In [4]:
using LinearAlgebra

In [5]:
# Example usage:
A = randn(7, 5)

7×5 Matrix{Float64}:
  0.667667  -0.511743   -0.44913    -1.54506    1.62512
  1.08274    0.752039   -0.638201    1.7185     0.286724
 -1.90628   -0.108286   -1.09271    -0.187644   0.446098
  0.259341   0.286976    4.40673     1.0601     0.123095
 -1.81144    2.16624    -1.7263     -0.90353   -1.1491
  0.57868    0.154887   -0.0426624  -2.14815    1.23479
  0.621095  -0.0431452  -1.78538    -0.918631  -0.208526

In [6]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Returns (inplace) 
    U: input matrix  
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    #U[cidx:end, cidx:end] = U[cidx:end, cidx:end] - 2 * v * (v' * U[cidx:end, cidx:end])
    #show(stdout, "text/plain", I - 2 * v * v')
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Returns (inplace) 
    H: input matrix  
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    
    #V[ridx:end, ridx+1:end] = V[ridx:end, ridx+1:end] - (V[ridx:end, ridx+1:n] * v) * (2 * v')
    #show(stdout, "text/plain", I - 2 * v * v')

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (B = Q_r * A * Q_l)
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
                #print("\ni = ",k,"\n")
                #show(stdout, "text/plain", B)
                #print("\n\n")
                #show(stdout, "text/plain", U * A * V)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U, B, V'
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [7]:
B, H = bidiagonalize(A);

In [8]:
show(stdout, "text/plain", B)

7×5 Matrix{Float64}:
 -3.05307  -2.06401       2.22045e-16   5.55112e-17   2.22045e-16
  0.0       4.13029      -2.66577      -1.11022e-16   2.22045e-16
  0.0       2.22045e-16  -3.19993      -1.44762      -6.93889e-18
  0.0       4.44089e-16   4.44089e-16   2.77522       0.813775
  0.0      -2.22045e-16   2.77556e-17   2.22045e-16  -1.58337
  0.0      -5.55112e-17   0.0           2.22045e-16   0.0
  0.0      -2.22045e-16  -2.77556e-17   2.22045e-16  -5.55112e-17

In [9]:
show(stdout, "text/plain", H)

7×5 Matrix{Float64}:
  0.780605    0.864051   -0.388301   -0.117979  -0.297857
  0.227157   -0.791085    0.969845    0.106133  -0.219401
 -0.399935    0.124116    0.926481    0.999887   0.0150592
  0.0544093   0.446561   -0.360289   -0.794226   1.0
 -0.380038   -0.270475   -0.0263207   0.400981   0.931267
  0.121406   -0.0461981   0.100784    0.39313    0.327251
  0.130305   -0.289953    0.0312182   0.232096  -0.16015

In [10]:
show(stdout, "text/plain", beta)

LoadError: UndefVarError: `beta` not defined

In [11]:
_, S1, _ = svd(B);

In [12]:
S1

5-element Vector{Float64}:
 5.653447077105473
 3.7909452105599586
 2.955013482670139
 1.9635073896516777
 1.4258866193788597

In [13]:
_, S2, _ = svd(A);

In [14]:
S2

5-element Vector{Float64}:
 5.653447077105471
 3.7909452105599555
 2.95501348267014
 1.9635073896516786
 1.4258866193788602

In [15]:
P_l, B, P_r = bidiagonalize(A, true);

In [16]:
# Checking correctness

In [17]:
show(stdout, "text/plain", opnorm(B - P_l * A * P_r', 1)/opnorm(A, 1))

5.420541948354418e-16

In [18]:
show(stdout, "text/plain", opnorm(P_r' * P_r -  I, 1))

1.461966616455773e-15

In [19]:
show(stdout, "text/plain", opnorm(P_l' * P_l - I, 1))

1.2433320166460106e-15

In [17]:
U, S, V = svd(B)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
7×5 Matrix{Float64}:
 -0.111962     -0.879563      0.138337     -0.438532      0.0488408
  0.544171      0.333746      0.163524     -0.746307      0.0936866
  0.830765     -0.334984     -0.0491754     0.437069     -0.0645631
  0.0342412    -0.0524668    -0.947505     -0.173304      0.261293
  0.000860157  -0.00394105   -0.232245     -0.172194     -0.957285
  4.38407e-17   7.66039e-18   1.99815e-19   8.17493e-17  -2.0901e-17
  3.6952e-19    5.71334e-18  -2.45056e-17  -2.10489e-18  -9.39519e-17
singular values:
5-element Vector{Float64}:
 4.975317941707024
 3.029726844353466
 1.9369461909792187
 1.3947000798229707
 1.092302772956615
Vt factor:
5×5 Matrix{Float64}:
  0.0515246  -0.271366    0.956612   -0.0926579   0.00367839
  0.664705   -0.706364   -0.227949    0.0845593  -0.010263
 -0.163526   -0.0657986   0.0789751   0.901771   -0.386655
  0.719924    0.640748    0.1511      0.0755615  -0.206423
 -0.102378   -0.111902   

In [18]:
P_l' * U

7×5 Matrix{Float64}:
  0.818411   -0.0375795   0.363408    0.0432272   -0.322577
 -0.191216   -0.218328    0.372298    0.604596     0.452173
 -0.387131    0.431531    0.610525    0.131656    -0.44774
 -0.233064   -0.674955    0.313052   -0.524787    -0.0722531
  0.243916    0.289015    0.369172   -0.31058      0.693977
  0.0243332  -0.287951    0.338288    0.00444256  -0.0455568
  0.171358   -0.377746   -0.0886577   0.493337    -0.0459517

In [19]:
V * P_r

5×5 Matrix{Float64}:
  0.0515246   -0.404646     0.897196    0.104413     0.133205
 -0.271366     0.235211     0.225751    0.224773    -0.87724
  0.956612     0.00301881  -0.0173622   0.0535212   -0.285864
 -0.0926579   -0.883484    -0.347517   -0.00926997  -0.300028
  0.00367839  -0.0196121   -0.151677    0.967277     0.202414

In [20]:
svd(A)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
7×5 Matrix{Float64}:
 -0.818411    0.0375795  -0.363408   -0.0432272    0.322577
  0.191216    0.218328   -0.372298   -0.604596    -0.452173
  0.387131   -0.431531   -0.610525   -0.131656     0.44774
  0.233064    0.674955   -0.313052    0.524787     0.0722531
 -0.243916   -0.289015   -0.369172    0.31058     -0.693977
 -0.0243332   0.287951   -0.338288   -0.00444256   0.0455568
 -0.171358    0.377746    0.0886577  -0.493337     0.0459517
singular values:
5-element Vector{Float64}:
 4.975317941707021
 3.029726844353465
 1.9369461909792178
 1.3947000798229707
 1.0923027729566164
Vt factor:
5×5 Matrix{Float64}:
 -0.0515246   0.547439   0.584519   0.367563    0.469991
 -0.664705   -0.507558   0.13115   -0.18433     0.499374
  0.163526    0.359147  -0.592447  -0.349101    0.609432
 -0.719924    0.534727  -0.228103  -0.0549008  -0.375143
  0.102378    0.166664   0.487968  -0.840259   -0.132645

In [23]:
function svd_golub_reinsh(B, maxiter = 80, eps = 1e-8)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step(n, B, U, V, p, q)
       # to be DONE 
    end
    
    n, m = size(B)
    U = zeros(m, m) + I
    V = zeros(n, n) + I
    maxiter_ = maxiter * (log2(n * m)) 
    
    for k = 1:maxiter
        # Step 1 of main loop
        for i = 1:n-1
            b[i, i + 1] = 0 ? abs(b[i, i + 1]) < eps * ( abs( b[i, i] ) + abs( b[i + 1, b + 1] ) ) :  b[i, i + 1]
        end
        
        # Step 2 of main loop
        # Find p and q such that matrix can be blocked as B11, B22 and B33 (diag) 
        #To be DONE
        
        # Step 3 of main loop
        # If q = n, return B (converged)
        
        # Step 4 of main loop
        
        
    end
    
end

svd_golub_reinsh (generic function with 3 methods)

In [21]:
function givens_rotation(v)
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end

givens_rotation (generic function with 1 method)

In [22]:
A = [1.0 3 4;4 -1 8;7 8 -1]

3×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0

In [23]:
G = givens_rotation(A[[1,2], 1])

2×2 Matrix{Float64}:
 -0.242536  -0.970143
  0.970143  -0.242536

In [24]:
A[[1,2], :] = G * A[[1,2], :];
A

3×3 Matrix{Float64}:
 -4.12311  0.242536  -8.73128
  0.0      3.15296    1.94029
  7.0      8.0       -1.0

In [25]:
G = givens_rotation(A[[1,3], 1])

2×2 Matrix{Float64}:
 0.507519  -0.86164
 0.86164    0.507519

In [26]:
A[[1,3], :] = G * A[[1,3], :];
A


3×3 Matrix{Float64}:
 -8.12404      -6.77003  -3.56965
  0.0           3.15296   1.94029
 -3.33067e-16   4.26913  -8.03075

In [27]:
G = givens_rotation(A[[2,3], 2])

2×2 Matrix{Float64}:
 -0.594089  -0.8044
  0.8044    -0.594089

In [28]:
A[[2,3], :] = G * A[[2,3], :];
A


3×3 Matrix{Float64}:
 -8.12404      -6.77003      -3.56965
  2.67919e-16  -5.30723       5.30723
  1.97871e-16   2.10348e-16   6.33174

In [29]:
qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 8.12404  6.77003   3.56965
 0.0      5.30723  -5.30723
 0.0      0.0       6.33174